In [1]:
import cv2
import torch
import numpy as np
from keras.models import load_model # type: ignore
from keras.applications.imagenet_utils import preprocess_input # type: ignore
import os

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)


In [2]:

class_names = [
    "Speed limit 20", "Speed limit 30", "Speed limit 50", "Speed limit 60",
    "Speed limit 70", "Speed limit 80", "End of speed limit 80", "Speed limit 100",
    "Speed limit 120", "No passing", "No passing for trucks", "Right-of-way at intersection",
    "Priority road", "Yield", "Stop", "No vehicles", "Vehicles over 3.5t prohibited",
    "No entry", "General caution", "Dangerous curve left", "Dangerous curve right",
    "Double curve", "Bumpy road", "Slippery road", "Road narrows on the right",
    "Road work", "Traffic signals", "Pedestrians", "Children crossing",
    "Bicycles crossing", "Beware of ice/snow", "Wild animals crossing",
    "End of all restrictions", "Turn right ahead", "Turn left ahead",
    "Ahead only", "Go straight or right", "Go straight or left", "Keep right",
    "Keep left", "Roundabout", "End of no passing", "End of no passing for trucks"
]

def preprocess_cnn(image):
    image = cv2.resize(image, (32, 32))
    image = image.astype('float32') / 255.0
    image = np.expand_dims(image, axis=0)
    return image

In [13]:
from ultralytics import YOLO

model = YOLO("yolov8_fine_tuned_v02.pt")

cap = cv2.VideoCapture(0)

cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    results = model(frame)

    annotated_frame = results[0].plot()

    cv2.imshow("Traffic Sign Detection", annotated_frame)

    # Press 'q' to quit
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()



0: 384x640 (no detections), 160.0ms
Speed: 4.2ms preprocess, 160.0ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 105.9ms
Speed: 6.0ms preprocess, 105.9ms inference, 2.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 102.4ms
Speed: 3.3ms preprocess, 102.4ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 78.7ms
Speed: 2.7ms preprocess, 78.7ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 67.2ms
Speed: 1.9ms preprocess, 67.2ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 69.7ms
Speed: 1.9ms preprocess, 69.7ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 63.7ms
Speed: 2.0ms preprocess, 63.7ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 65.5ms
Speed: 2.1ms preprocess, 65

In [3]:
from ultralytics import YOLO

model = YOLO("yolov8_fine_tuned_v02.pt")
model.info()

Model summary: 129 layers, 3,019,233 parameters, 0 gradients, 8.2 GFLOPs


(129, 3019233, 0, 8.2393088)

In [ ]:
print(model.model)

In [9]:

image_folder = "GTSDB/images/train"
labels_folder = "GTSDB/labels/train"

output_file = "v3_test_pred.txt"

# Open output file for writing
with open(output_file, "w") as f:
    # Loop over each image
    for img_name in os.listdir(image_folder):
        if img_name.lower().endswith((".jpg", ".png", ".jpeg")):
            img_path = os.path.join(image_folder, img_name)
            results = model.predict(source=img_path, save=False, conf=0.25, verbose=False)

            # Get class IDs for detections in current image
            class_ids = results[0].boxes.cls.tolist()

            # Write to file: image_name class_id class_id ...
            f.write(f"{img_name} " + " ".join(str(int(cls_id)) for cls_id in class_ids) + "\n")



In [10]:
with open(output_file, "r") as f:
    lines = f.readlines()
pred = []
for line in lines:
    parts = line.strip().split(' ')
    img_name = parts[0]
    if len(parts) > 1:
        class_ids = [int(cls_id) for cls_id in parts[1:]]
    else:
        class_ids = [99]
    if class_ids:
        pred.append((img_name, class_ids))
actual = []
for file in os.listdir(labels_folder):
    with open(os.path.join(labels_folder, file), "r") as f:
        content = f.readlines()
        content = [line.strip() for line in content if line.strip()]
        if content:
            class_id = []
            for line in content:
                class_id.append(int(line.split()[0]))
            actual.append((file, class_id))   
sum = 0
count = 0

for i in range(len(actual)):
    count += (max(len(actual[i][1]), len(pred[i][1])))

for i in range(len(actual)):
    for j in pred[i][1]:
        if j in actual[i][1]:
            sum += 1
accuracy = sum / count
accuracy

0.03830480847595762

In [7]:
len(pred), len(actual)

(242, 741)